In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report

input_data = pd.read_csv('../../DS2/Common_Code/one_hot_encoded_disease_input.csv')
output_data = pd.read_csv('../../DS2/Common_Code/disease_output.csv')

merged_dataset = pd.merge(input_data, output_data, on='dog_id', how='inner')
# merged_dataset.to_csv('merged_dataset.csv', index=False)

print(merged_dataset)

       dog_id  102.0  202.0  304.0  305.0  306.0  307.0  412.0  423.0  426.0  \
0      100001  False  False   True   True   True  False  False  False  False   
1      100004  False  False  False  False  False  False  False  False  False   
2      100005   True  False  False  False  False  False  False  False  False   
3      100006  False  False   True  False  False  False  False  False  False   
4      100007  False  False  False  False  False  False  False  False  False   
...       ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
26697   99992  False  False  False  False  False  False  False  False  False   
26698   99995  False  False  False  False  False  False  False  False  False   
26699   99998  False  False  False  False  False  False  False  False  False   
26700   99999  False  False  False  False  False  False  False  False  False   
26701   82743  False  False  False  False  False  False  False  False  False   

       ...  hs_health_conditions_reprod

In [2]:
from sklearn.model_selection import KFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, roc_auc_score
import numpy as np

input_features = merged_dataset.drop(['dog_id'] + output_data.columns[1:].tolist(), axis=1)
original_condition = 'hs_health_conditions_eye'

# List of health conditions to replace
replacement_conditions = ['hs_health_conditions_ear', 'hs_health_conditions_oral', 
                           'hs_health_conditions_skin', 'hs_health_conditions_cardiac', 
                           'hs_health_conditions_respiratory', 'hs_health_conditions_gastrointestinal', 
                           'hs_health_conditions_liver', 'hs_health_conditions_kidney', 
                           'hs_health_conditions_reproductive', 'hs_health_conditions_orthopedic', 
                           'hs_health_conditions_neurological', 'hs_health_conditions_endocrine', 
                           'hs_health_conditions_hematologic', 'hs_health_conditions_immune', 
                           'hs_health_conditions_infectious_disease']

# Replace the column name in a loop
for replacement_condition in replacement_conditions:
    y = merged_dataset[original_condition.replace(original_condition, replacement_condition)]
    # 5 K-fold cross validation
    kf = KFold(n_splits=5, random_state=42, shuffle=True)

    # train and evaluate for every fold
    for train_index, test_index in kf.split(input_features):
        X_train, X_test = input_features.iloc[train_index], input_features.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # use naive bayes as classifier
        model = MultinomialNB()
        model.fit(X_train, y_train)

        # predict
        y_pred = model.predict(X_test)

        # accuracy
        accuracy = accuracy_score(y_test, y_pred)
        print("Accuracy for 'hs_health_conditions_reproductive':", accuracy)

        # predict probabilities
        y_pred_prob = model.predict_proba(X_test)

        # compute AUC for binary or multiclass classification
        if len(np.unique(y)) == 2:  # binary classification
            auc = roc_auc_score(y_test, y_pred_prob[:, 1])
            print("AUC for 'hs_health_conditions_reproductive':", auc)
        else:  # multiclass classification
            auc = roc_auc_score(y_test, y_pred_prob, multi_class='ovr')
            print("AUC for 'hs_health_conditions_reproductive':", auc)

Accuracy for 'hs_health_conditions_reproductive': 0.9232353491855457
AUC for 'hs_health_conditions_reproductive': 0.6623158620854697
Accuracy for 'hs_health_conditions_reproductive': 0.9222991949073207
AUC for 'hs_health_conditions_reproductive': 0.6397945403237482
Accuracy for 'hs_health_conditions_reproductive': 0.9269662921348315
AUC for 'hs_health_conditions_reproductive': 0.7515732005889821
Accuracy for 'hs_health_conditions_reproductive': 0.9282771535580524
AUC for 'hs_health_conditions_reproductive': 0.6827891622614151
Accuracy for 'hs_health_conditions_reproductive': 0.9310861423220974
AUC for 'hs_health_conditions_reproductive': 0.7684184177729622
Accuracy for 'hs_health_conditions_reproductive': 0.9780939898895338
AUC for 'hs_health_conditions_reproductive': 0.7713481414637844
Accuracy for 'hs_health_conditions_reproductive': 0.9734132184984086
AUC for 'hs_health_conditions_reproductive': 0.8144883221021255
Accuracy for 'hs_health_conditions_reproductive': 0.9715355805243445


ValueError: Number of classes in y_true not equal to the number of columns in 'y_score'